In [1]:
import serial
import time 
from datetime import datetime
#to get port number type 'ls /dev/tty.*' in terminal

In [4]:
def main():
    port = '/dev/tty.usbmodem1101'  # Replace with your Arduino's port
    baudrate = 115200
    solenoidCount = 0
    # Initialize serial connection
    ser = serial.Serial(port, baudrate, timeout=1)
    time.sleep(2)  # Wait for the connection to be established

    # Send start command to Arduino
    ser.write(b'S')
    print('Sent start command to Arduino.')

    try:
        # Read data from Arduino
        while True:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').rstrip()
                print(f'Arduino: {line}')
                if line == 'Solenoid Activated':
                    solenoidCount+=1
                if solenoidCount >10:
                    break  # Exit loop after solenoid activation (modify as needed)

            # Check for stop condition or other conditions as needed
            # e.g., send stop command 'X' to Arduino
            # ser.write(b'X')
            # break
    except KeyboardInterrupt:
        print("Interrupted by user")
        ser.write(b'X')  # Send stop command to Arduino
    finally:
        # Close serial connection
        ser.close()

# Ensure the main function is called if this script is executed directly
if __name__ == '__main__':
    main()


Sent start command to Arduino.
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 1
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 1
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 1
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: lick: 0
Arduino: Sol pin state: 0
Arduino: bar: 0
Arduino: 

In [ ]:
#figure out how to do keyboard interrrupt and just keep that, keyboard module seems to fuck things up. 

In [3]:
def main():
    port = '/dev/tty.usbmodem1101'  # Replace with your Arduino's port
    baudrate = 115200  # Ensure this matches the baud rate in the Arduino sketch

    # Initialize serial connection
    ser = serial.Serial(port, baudrate, timeout=1)
    time.sleep(2)  # Wait for the connection to be established
    #dont send commands to other things (2p,camera) till after this either 

    # Send start command to Arduino
    start_time = time.time() #use for duration based arduino script start/stop
    ser.write(b'S')
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    print(f'{timestamp} - Sent start command to Arduino.')
    
    try:
        # Read data from Arduino
        while True:
            if ser.in_waiting > 0:
                try:
                    line = ser.readline().decode('utf-8', errors='ignore').rstrip()
                    timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
                    print(f'{timestamp} - Arduino: {line}')
                except UnicodeDecodeError:
                    print("Failed to decode serial data")
                    continue

            # Check if 20 seconds have passed to send the stop command
            if time.time() - start_time >= 20:
                ser.write(b'X')
                print('Sent stop command to Arduino.')
                break

    except KeyboardInterrupt:
        ser.write(b'X')
        print("Interrupted by user. Sent stop command to Arduino.")
    finally:
        # Close serial connection
        ser.close()

# Ensure the main function is called if this script is executed directly
if __name__ == '__main__':
    main()


2024-08-02 11:52:59.627 - Sent start command to Arduino.
11:52:59.635 - Arduino: Arduino doing things, Arduino started
11:52:59.635 - Arduino: bar: 0
11:52:59.635 - Arduino: lick: 0
11:52:59.643 - Arduino: bar: 8
11:52:59.647 - Arduino: lick: 2
11:52:59.655 - Arduino: bar: 4
11:52:59.655 - Arduino: lick: 3
11:52:59.663 - Arduino: bar: 7
11:52:59.663 - Arduino: lick: 2
11:52:59.671 - Arduino: bar: 5
11:52:59.671 - Arduino: lick: 2
11:52:59.679 - Arduino: bar: 8
11:52:59.683 - Arduino: lick: 2
11:52:59.692 - Arduino: bar: 7
11:52:59.692 - Arduino: lick: 2
11:52:59.700 - Arduino: bar: 7
11:52:59.700 - Arduino: lick: 2
11:52:59.708 - Arduino: bar: 7
11:52:59.708 - Arduino: lick: 3
11:52:59.716 - Arduino: bar: 5
11:52:59.720 - Arduino: lick: 2
11:52:59.729 - Arduino: bar: 7
11:52:59.729 - Arduino: lick: 3
11:52:59.737 - Arduino: bar: 5
11:52:59.737 - Arduino: lick: 2
11:52:59.745 - Arduino: bar: 7
11:52:59.745 - Arduino: lick: 2
11:52:59.757 - Arduino: bar: 5
11:52:59.757 - Arduino: lick: 2